In [1]:
using OceananigansLagrangianFilter
using Printf
using JLD2
using Oceananigans.Units: Time
using CairoMakie
using CUDA
using Oceananigans.TimeSteppers: reset!
using GeoStats: IDW


In [2]:
# User defined options

original_data_filename = joinpath(@__DIR__, "SW_vort_wave_only_tracer.jld2")
T_start = 0
T_end = 40

arch = GPU()

# Set the output period
T_out = 0.1

# Set filter order and cut-off frequency
# Amplitude of frequency response of filter will be squared Butterworth order 2^N
N = 2 
freq_c = 1

# Define variables to filter
original_var_names = ("ω","T")

# Define velocities to use for filtering
velocity_names = ("u","v")

# Set filtering parameters (this is for Butterworth-type, could define others here)
filter_params = set_BW_filter_params(N=N,freq_c=freq_c)

# Set the time step for the simulation
Δt = 1e-3

# Decide whether to solve for and output maps to generalised Lagrangian mean
map_to_mean = true

# Name output files
forward_output_filename = joinpath(@__DIR__, "forward_LF_freqc_1_tracer_wave_only_debug_forcing.jld2")
backward_output_filename = joinpath(@__DIR__, "backward_LF_freqc_1_tracer_wave_only_debug_forcing.jld2")
combined_output_filename = joinpath(@__DIR__, "combined_LF_freqc_1_tracer_wave_only_debug_forcing.jld2")
# Manipulate data on disk to have correct order 
T = set_data_on_disk!(original_data_filename, direction="forward", T_start = T_start, T_end = T_end)

Current direction is backward
Reversing order of data
Reversed order of ω
Reversed order of and switched sign of u
Reversed order of and switched sign of v
Reversed order of T
Reversed order of t_simulation
Reversed order of and shifted t
New direction is forward


40

In [3]:
# Load in saved data from simulation
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))
println("Loaded data from $original_data_filename")

# Create the original variables - these will be auxiliary fields in the model
original_vars = create_original_vars(original_var_names, grid)
println("Created original variables: ", original_vars)

# Create the filtered variables - these will be tracers in the model
filtered_vars = create_filtered_vars(original_var_names, velocity_names, filter_params, map_to_mean=map_to_mean)
println("Created filtered variables: ", filtered_vars)

# Create forcing for these filtered variables
forcing = create_forcing(filtered_vars, original_var_names, velocity_names, filter_params)
println("Created forcing for filtered variables ")

# Define model 
model = LagrangianFilter(;grid, tracers = filtered_vars, auxiliary_fields = original_vars, forcing = forcing)
println("Created model")

# Define our outputs # 
filtered_outputs = create_output_fields(model, original_var_names, velocity_names, filter_params)
println("Defined outputs")

# Define the filtering simulation 
simulation = Simulation(model, Δt = Δt, stop_time = T) 
println("Defined simulation")

# Tell the simulation to use the saved data
simulation.callbacks[:update_input_data] = Callback(update_input_data!, parameters = (velocities = saved_velocities, original_vars = saved_original_vars))

# Add a progress monitor
function progress(sim)
    @info @sprintf("Simulation time: %s, max(|u|):%.2e \n", 
                   prettytime(sim.model.clock.time), 
                   maximum(abs, model.velocities.u))             
     return nothing
 end

simulation.callbacks[:progress] = Callback(progress, IterationInterval(50))

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = forward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

run!(simulation)

Loaded data from /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/pure_wave_SW/SW_vort_wave_only_tracer.jld2
Created original variables: NamedTuple with 2 Fields on 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo:
├── T: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
└── ω: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
Created filtered variables: (:ωC1, :ωC2, :TC1, :TC2, :xi_u_C1, :xi_u_C2, :xi_v_C1, :xi_v_C2, :ωS1, :ωS2, :TS1, :TS2, :xi_u_S1, :xi_u_S2, :xi_v_S1, :xi_v_S2)
Deleted a bit for debugging
Deleted a bit for debugging
Deleted a bit for debugging
Deleted a bit for debugging
Created forcing for filtered variables 
Created model
Defined outputs
Defined simulation


JLD2Writer scheduled on TimeInterval(100 ms):
├── filepath: forward_LF_freqc_1_tracer_wave_only_debug_forcing.jld2
├── 6 outputs: (xi_u, xi_v, T, ω, ω_filtered, T_filtered)
├── array_type: Array{Float32}
├── including: [:grid]
├── file_splitting: NoFileSplitting
└── file size: 50.3 KiB

In [4]:
run!(simulation)

[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):8.33e-01 
[ Info:     ... simulation initialization complete (3.058 minutes)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (16.969 seconds).
[ Info: Simulation time: 50.000 ms, max(|u|):8.38e-01 
[ Info: Simulation time: 100.000 ms, max(|u|):9.05e-01 
[ Info: Simulation time: 148.000 ms, max(|u|):9.03e-01 
[ Info: Simulation time: 198.000 ms, max(|u|):9.34e-01 
[ Info: Simulation time: 248.000 ms, max(|u|):9.03e-01 
[ Info: Simulation time: 298.000 ms, max(|u|):9.13e-01 
[ Info: Simulation time: 348.000 ms, max(|u|):8.86e-01 
[ Info: Simulation time: 398.000 ms, max(|u|):9.11e-01 
[ Info: Simulation time: 448.000 ms, max(|u|):9.05e-01 
[ Info: Simulation time: 498.000 ms, max(|u|):9.49e-01 
[ Info: Simulation time: 548 ms, max(|u|):9.54e-01 
[ Info: Simulation time: 598.000 ms, max(|u|):1.00e+00 
[ Info: Simulation time: 648.000 ms, max(|u|):9.91e-01 
[ Info: Simulation t

In [5]:

# Now, run it backwards. Switch the data direction on disk
T = set_data_on_disk!(original_data_filename, direction="backward", T_start = T_start, T_end = T_end)

# Reload the saved data 
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))

# Reinitialize tracers to zero (for consistency)
kwargs = (; (tracer => 0 for tracer in keys(model.tracers))...)
set!(model; kwargs...)  

reset!(model.clock)

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = backward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

# And run the backward simulation.
run!(simulation)


Current direction is forward
Reversing order of data
Reversed order of ω
Reversed order of and switched sign of u
Reversed order of and switched sign of v
Reversed order of T
Reversed order of t_simulation
Reversed order of and shifted t
New direction is backward


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):9.12e-01 
[ Info:     ... simulation initialization complete (71.441 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (8.922 ms).
[ Info: Simulation time: 50.000 ms, max(|u|):8.63e-01 
[ Info: Simulation time: 100.000 ms, max(|u|):8.67e-01 
[ Info: Simulation time: 148.000 ms, max(|u|):8.63e-01 
[ Info: Simulation time: 198.000 ms, max(|u|):9.12e-01 
[ Info: Simulation time: 248.000 ms, max(|u|):9.14e-01 
[ Info: Simulation time: 298.000 ms, max(|u|):9.58e-01 
[ Info: Simulation time: 348.000 ms, max(|u|):9.64e-01 
[ Info: Simulation time: 398.000 ms, max(|u|):1.00e+00 
[ Info: Simulation time: 448.000 ms, max(|u|):9.80e-01 
[ Info: Simulation time: 498.000 ms, max(|u|):9.83e-01 
[ Info: Simulation time: 548 ms, max(|u|):9.42e-01 
[ Info: Simulation time: 598.000 ms, max(|u|):9.66e-01 
[ Info: Simulation time: 648.000 ms, max(|u|):9.83e-01 
[ Info: Simulation time: 698.0

In [6]:
sum_forward_backward_contributions!(combined_output_filename, forward_output_filename, backward_output_filename, T, velocity_names, original_var_names)

Combined forward and backward contributions into /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/pure_wave_SW/combined_LF_freqc_1_tracer_wave_only_vary_y.jld2


In [7]:
# Next step is to interpolate onto the correct grid
using GeoStats: Polynomial, NN, Kriging, GaussianVariogram

maxneighbors = 40
npad = 5
model = Kriging(GaussianVariogram(range=2*2*pi/256))
regrid_to_mean_position!(combined_output_filename, original_var_names, velocity_names, model, maxneighbors,npad)



0.0%┣                                              ┫ 0/401 [00:00<00:00, -0s/it]
0.2%┣                                          ┫ 1/401 [00:11<Inf:Inf, InfGs/it]
0.5%┣▏                                          ┫ 2/401 [00:18<01:59:26, 18s/it]
0.7%┣▎                                          ┫ 3/401 [00:25<01:22:55, 13s/it]
1.0%┣▍                                          ┫ 4/401 [00:32<01:10:42, 11s/it]
1.2%┣▌                                          ┫ 5/401 [00:39<01:04:31, 10s/it]
1.5%┣▋                                           ┫ 6/401 [00:46<01:00:47, 9s/it]
1.7%┣▉                                              ┫ 7/401 [00:53<58:14, 9s/it]
2.0%┣█                                              ┫ 8/401 [01:00<56:24, 9s/it]
2.2%┣█                                              ┫ 9/401 [01:07<54:59, 8s/it]
2.5%┣█▏                                            ┫ 10/401 [01:12<52:26, 8s/it]
2.7%┣█▎                                            ┫ 11/401 [01:15<49:04, 8s/it]
3.0%┣█▍                     

In [10]:
# Animate
timeseries1 = FieldTimeSeries(combined_output_filename, "T")
timeseries2 = FieldTimeSeries(combined_output_filename, "T_filtered")
timeseries3 = FieldTimeSeries(combined_output_filename, "T_filtered_regrid")
times = timeseries1.times

set_theme!(Theme(fontsize = 20))
fig = Figure(size = (1000, 500))

axis_kwargs = (xlabel = "x",
               ylabel = "y",
               limits = ((0, 2π), (0, 2π)),
               aspect = AxisAspect(1))

ax1 = Axis(fig[2, 1]; title = "Raw vorticity", axis_kwargs...)
ax2 = Axis(fig[2, 2]; title = "Filtered vorticity", axis_kwargs...)
ax3 = Axis(fig[2, 3]; title = "Filtered T", axis_kwargs...)


n = Observable(1)
Observable(1)

var1 = @lift timeseries1[$n]
var2 = @lift timeseries2[$n]
var3 = @lift timeseries3[$n]

heatmap!(ax1, var1; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax2, var2; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax3, var3; colormap = :balance, colorrange = (0, 1))


title = @lift "t = " * string(round(times[$n], digits=2))
Label(fig[1, 1:3], title, fontsize=24, tellwidth=false)

fig

frames = 1:length(times)

@info "Making an animation"

CairoMakie.record(fig, "LF_movie_freq_c_1_wave_only_vary_y.mp4", frames, framerate=24) do i
    n[] = i
end

[ Info: Making an animation


"LF_movie_freq_c_1_wave_only_vary_y.mp4"

In [8]:
jld2_to_netcdf(combined_output_filename, joinpath(@__DIR__, "combined_output_freq_c_1_wave_only_vary_y.nc"))

closed Dataset

In [ ]:
jld2_to_netcdf(forward_output_filename, joinpath(@__DIR__, "forward_output.nc"))
jld2_to_netcdf(backward_output_filename, joinpath(@__DIR__, "backward_output.nc"))

closed Dataset

In [5]:
A = Matrix([1 2 3; 4 5 6; 7 8 9])
size(A)

(3, 3)

In [ ]:
combined_output_filename